In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import os
import analysis

2024-08-15 16:39:54.356116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 16:39:54.366765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 16:39:54.370019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 16:39:54.377803: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-15 16:39:54.905938: W tensorflow/compiler/tf2

In [2]:
x_test, y_test = analysis.LoadData('data/test/',['normal', 'pneumonia'], img_size = 250)

data/test/normal
data/test/pneumonia


In [3]:
model = load_model('CNN1.keras')

I0000 00:00:1723736398.879146   23234 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723736398.901207   23234 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723736398.904677   23234 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723736398.908728   23234 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [4]:
predictions = model.predict(x_test)

I0000 00:00:1723736399.997809   23334 service.cc:146] XLA service 0x71c32c002d20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723736399.997825   23334 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-08-15 16:40:00.002379: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-15 16:40:00.036999: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


35/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

I0000 00:00:1723736400.604789   23334 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


## Model evaluation

#### Accuracy

In [5]:
y_pred = (predictions > 0.5).flatten().astype(int)
accuracy = np.mean(y_pred == y_test)
print(f'accuracy = {accuracy:.4f}')

accuracy = 0.9298


Using tensorflow:

In [6]:
tf_y_pred = tf.cast(predictions > 0.5, tf.int32)
tf_y_pred = tf.reshape(tf_y_pred, [-1]) #Reshape to use with y_test
tf_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf_y_pred, y_test), tf.float32))
print(f'tf accuracy = {tf_accuracy.numpy():.4f}')

tf accuracy = 0.9298


#### Precision, Recall, F1

In [7]:
precision, recall, f1 = analysis.MetricsCalc(y_pred, y_test)

In [8]:
print(f'Precision = {precision:.4f}')
print(f'Recall = {recall:.4f}')
print(f'F1 Score = {f1:.4f}')

Precision = 0.9731
Recall = 0.9298
F1 Score = 0.9510
